<a href="https://colab.research.google.com/github/Clinda02/financial_data_analysis/blob/main/Finance_news_scarp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개요
* 네이버 증권의 `뉴스` 탭의 최신 10일치 기사를 수집하여 데이터 프레임으로 생성, 데이터프레임을 `article.csv` 로 저장
* 사이트 주소 : https://finance.naver.com/news/mainnews.naver?date=2024-10-25


-----

**네이버 증권 뉴스 탭**
* 날짜별로 페이지가 구분되어 있고, 날짜를 변경할 경우, URL의 date의 속성값 변경
  * `https://finance.naver.com/news/mainnews.naver?date=2024-10-25`

* 각 날짜에는 페이지네이션으로 페이지가 변경됨
  * `https://financenaver.com/news/mainnews.naver?date=2024-10-25&page=3`
* 페이지네이션은 고려하지 않고, 첫 페이지에 있는 데이터만 수집

**데이터 프레임 구조**
* `기사제목, 본문URL, 기사본문, 날짜` 4개 칼럼으로 구성

# 신문기사 HTML 불러오기

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# 뉴스 스크랩
url = "https://finance.naver.com/news/mainnews.naver?date=2024-10-31"
response = requests.get(url)

soup = BeautifulSoup(response.text, 'lxml')

- 데이터 프레임 구조

  - **기사제목**:  `<dd class="articleSubject">`안에 텍스트
  - **본문 URL**: `<dd class="articleSubject">` 안에 `<a>` 태그
  - **기사 본문**: `<div id="newsct_article" class="newsct_article _article_body">`
  - **날짜**: URL에서 가져오기

In [2]:
# 기사제목 리스트 가져오기
titles = soup.find_all('dd', {'class':'articleSubject'})
titles[0]

<dd class="articleSubject">
<a href="/news/news_read.naver?article_id=0005873734&amp;office_id=018&amp;mode=mainnews&amp;type=&amp;date=2024-10-31&amp;page=1">공개매수 하면서 유증 계획 숨겼나…‘고의누락’ 의혹 휩싸인 고려아연[마켓인]</a>
</dd>

In [3]:
titles[0].text.strip()

'공개매수 하면서 유증 계획 숨겼나…‘고의누락’ 의혹 휩싸인 고려아연[마켓인]'

# 상세 뉴스 본문 읽어오기

## 상세 뉴스 URL 읽어오기

In [4]:
# 첫번째 기사의 상세페이지 URL
titles[0].find('a').get('href')

'/news/news_read.naver?article_id=0005873734&office_id=018&mode=mainnews&type=&date=2024-10-31&page=1'

In [5]:
news_url = 'https://finance.naver.com' + titles[0].find('a').get('href')
news_url

'https://finance.naver.com/news/news_read.naver?article_id=0005873734&office_id=018&mode=mainnews&type=&date=2024-10-31&page=1'

In [6]:
# 상세 뉴스 페이지
news_response = requests.get(news_url)
news_soup = BeautifulSoup(news_response.text, 'lxml')
news_soup

<html><head><script>top.location.href='https://n.news.naver.com/mnews/article/018/0005873734';</script>
</head></html>

redirect 주소를 반환하고 있으므로, redirecet 주소를 사용하여 다시 상세 뉴스 페이지에 접근

In [7]:
news_url2 = news_soup.find('script').text.split("'")[1]

## 상세 뉴스 페이지 읽어오기

In [8]:
# 상세 뉴스페이지 재호출
news_response2 = requests.get(news_url2)

news_soup = BeautifulSoup(news_response2.text, 'lxml')

## 상세 뉴스페이지 본문 파싱하기

In [9]:
# 뉴스 본문
# 기사 본문: <div id="newsct_article" class="newsct_article _article_body">
news_soup.find('div',{'class', 'newsct_article _article_body'}).text.strip()

'금감원, ‘위계 활용 부정거래’ 가능성 제기공개매수 주관사 미래에셋도 검사 착수향후 10일 내 공시 정정 여부 결정유상증자 철회·발행가액 정정 가능성\n\n\n\n최윤범 고려아연 회장이 기자회견에서 발언하고 있다 (사진=연합뉴스)[이데일리 마켓in 허지은 기자] 2조5000억원 규모 유상증자를 결의한 고려아연이 ‘고의 누락’ 의혹에 휩싸였다. 앞서 진행된 자사주 공개매수 기간과 유상증자 검토 기간이 겹치면서다. 고려아연의 공개매수와 유상증자 주관 업무를 모두 맡은 미래에셋증권과, 이사회에서 해당 안건을 결의한 이사진들이 이를 알고도 숨겼다면 중대한 사항의 누락, 나아가 위계 활용 부정거래 소지가 있다고 금융당국은 판단했다. 함용일 금융감독원 부원장은 31일 진행한 긴급 브리핑에서 “고려아연의 불공정거래 조사와 관련해 부정거래 행위 등을 조사하고 있다”며 “고려아연의 재무 계획상 공개매수와 유상증자 두 가지 사실을 다 알고 하나씩 내보이는 것은 (중대한 사항을) 누락하는 것이다. 고려아연 증자의 목적, 배경, 회사와 기존 주주에 미치는 영향, 증자가 주주가치 제고에 부합되는지 여부 등을 철저히 심사할 것”이라고 밝혔다. 금감원에 따르면 고려아연이 전날 발표한 유상증자 결정은 지난 14일부터 29일까지 주관사 미래에셋증권의 기업실사 후 이사회 결의를 통해 확정됐다. 14일은 MBK·영풍 연합이 공개매수를 마감한 날이다. 앞서 고려아연은 지난 2일부터 23일까지 자사주 공개매수를 진행했는데, 해당 기간과 유상증자를 위한 기업실사 기간이 겹친다. 금감원은 고려아연이 공개매수 기간 동안 유상증자를 계획했으면서 이를 알리지 않은 경우 중대한 사항을 누락했다고 봤다. 함 부원장은 “이사회가 이를 다 아는 상태에서 순차적으로 진행한 거라면 공개매수 신고서에 중대한 사항이 빠졌거나 허위 위계 부정거래 소지가 다분하다”며 “주주들이 공개매수에 응할지 의사결정할 때 이사들이 대량의 증자 소식을 알았다면 부정거래 행위 등 공개매수 허위 기재 문제점을 따져보지 않을 수 없다”고 강

# 반복문 사용하여 함수로 생성하기

In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime
import time

In [11]:
 def get_news_items(html):

  # 기사 리스트 가져오기
  titles = html.find_all('dd', {'class' :'articleSubject'})

  title_list = []
  url_list =[]
  content_list = []

  for t in titles:
    # 뉴스 기사 제목
    title = t.text.strip()
    # print(f"제목 : {title}")

    # 뉴스 기사의 상세페이지 url
    news_url = 'https://finance.naver.com' + t.find('a').get('href')
    # print(f"url : {news_url}")

    # 상세페이지 request
    news_response = requests.get(news_url)
    news_soup = BeautifulSoup(news_response.text, 'lxml')

    # 리다이렉트 주소 파싱
    news_url2 = news_soup.find('script').text.split("'")[1]

    # 상세페이지 request
    news_response2 = requests.get(news_url2)
    news_soup = BeautifulSoup(news_response2.text, 'lxml')

    # 뉴스 본문
    content = news_soup.find('div', {'class':'newsct_article _article_body'}).text.strip()
    # print(f"본문 : {content}")

    # 리스트 값 저장
    title_list.append(title)
    url_list.append(news_url2)
    content_list.append(content)

  # 데이터프레임 저장
  df = pd.DataFrame({
        '기사제목' : title_list,
        '본문url' : url_list,
        '기사본문' : content_list
    })
  return df


In [ ]:
news_df = pd.DataFrame()

for n in range(10):
  # 최근 n일치 데이터 지정
  date = datetime.date.today() - datetime.timedelta(days = n)

  url = f"https://finance.naver.com/news/mainnews.naver?date={date}"
  response = requests.get(url)

  # 상태 코드 확인
  if response.status_code == 200:
    print(f"{date} 페이지 접근 성공")

    soup = BeautifulSoup(response.text, 'lxml')
    time.sleep(1)

    # 해당일 신문기사 스크랩 데이터프레임 반환
    temp_df = get_news_items(soup)
    temp_df['date']= date
    news_df = pd.concat([news_df, temp_df], ignore_index =True).copy()

  else :
    print(f"{date} 페이지 접근 실패 - 상태코드 : {response.status_code}")

2024-10-31 페이지 접근 성공
2024-10-30 페이지 접근 성공
2024-10-29 페이지 접근 성공
2024-10-28 페이지 접근 성공
2024-10-27 페이지 접근 성공
2024-10-26 페이지 접근 성공
2024-10-25 페이지 접근 성공
2024-10-24 페이지 접근 성공
2024-10-23 페이지 접근 성공
2024-10-22 페이지 접근 성공


In [13]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   기사제목    40 non-null     object
 1   본문url   40 non-null     object
 2   기사본문    40 non-null     object
 3   date    40 non-null     object
dtypes: object(4)
memory usage: 1.4+ KB


In [14]:
news_df.head()

,기사제목,본문url,기사본문,date
0,공개매수 하면서 유증 계획 숨겼나…‘고의누락’ 의혹 휩싸인 고려아연[마켓인],https://n.news.naver.com/mnews/article/018/000...,"금감원, ‘위계 활용 부정거래’ 가능성 제기공개매수 주관사 미래에셋도 검사 착수향후...",2024-10-31
1,"고려아연, 5일만에 `황제주` 반납…시총순위 7계단↓",https://n.news.naver.com/mnews/article/029/000...,2.5조 유상증자 결정 이후 연이틀 급락\n\n\n\n[연합뉴스 제공] 전날 ...,2024-10-31
2,고려아연에 칼 뽑은 금감원… 함용일 부원장 “당국으로서 결코 간과할 수 없는 문제”,https://n.news.naver.com/mnews/article/366/000...,"31일 기자브리핑 개최금감원, 고려아연 이사회 결정 강한 의구심공개매수·유상증자 도...",2024-10-31
3,"증자할 거 알면서도 공개매수?…금감원, 고려아연 불공정 여부 살핀다",https://n.news.naver.com/mnews/article/648/000...,"함용일 부원장, 31일 고려아연 등 자본시장현안 브리핑 공개매수와 유상증자 동시 추...",2024-10-31
4,"혼돈의 코스피, 연 이틀 약세…""애플·아마존 실적 주목""",https://n.news.naver.com/mnews/article/008/000...,내일의 전략\n\n\n\n/일러스트=임종철 디자인기자미국 대선과 실적 등 증시 불확...,2024-10-31


In [15]:
news_df.tail()

,기사제목,본문url,기사본문,date
35,을지로 유안타증권 빌딩 5920억 PF 성공,https://n.news.naver.com/mnews/article/014/000...,"캡스톤운용, 2022년 인수 후 쾌거 \n\n\n\n유안타증권 본사 전경 유안타증권...",2024-10-30
36,"강남 르메르디앙 개발, 내년 하반기 '첫삽'…대출 9500억, 내년 6월 만기",https://n.news.naver.com/mnews/article/018/000...,웰스어드바이저스·현대건설·마스턴운용·메리츠 '컨소''창의·혁신디자인' 시범사업지…서...,2024-10-30
37,"영풍·MBK ""고려아연 유증 저지 위한 모든 법적수단 강구""",https://n.news.naver.com/mnews/article/015/000...,김광일 MBK 파트너스 부회장(가운데). 사진=뉴스1최윤범 고려아연 회장 측이 3...,2024-10-30
38,"""미장은 장기투자용, 국장은 단타용""…MZ들은 국장을 왜 떠나나",https://n.news.naver.com/mnews/article/421/000...,"[NIF2024]MZ토론회…국장은 불확실·불안정, 미장은 성장성 주목 국장 대한 애...",2024-10-30
39,진흙탕 싸움된 고려아연 사태 … 금감원 칼 뺐다,https://n.news.naver.com/mnews/article/009/000...,"금감원, 고려아연 유증 제동시세조종 정황도 포착한 당국불공정행위 여부 고강도 조사주...",2024-10-30


#  CSV 파일로 저장

In [20]:
news_df.to_csv('article.csv')
print('CSV 파일 저장 완료')

CSV 파일 저장 완료
